In [1]:
%matplotlib inline
''' Junhong Kim
Ph.D. Student in Industrial Management Engineering
Korea University, Seoul, Republic of Korea
Mobile Phone +82 10 3099 3004
E-mail    junhongkim@korea.ac.kr
Data Science and Business Analytics Lab
LinkedIn https://www.linkedin.com/in/charcoalgrey/
Lab Homepage http://dsba.korea.ac.kr'''

import numpy as np
import os
import copy
import pandas as pd
from sklearn.metrics import confusion_matrix
from scipy.stats import multivariate_normal
from sklearn.neural_network import MLPRegressor
from sklearn import mixture
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import collections
import warnings
warnings.filterwarnings("ignore")
print("Module Ready!")

Module Ready!


### [1] 파일을 불러온다

In [2]:
Root_Directory = os.path.join(os.getcwd(),"data")

In [3]:
TR_Data = np.array(pd.read_csv(os.path.join(Root_Directory,"Cancer_TR.csv")))
VL_Data = np.array(pd.read_csv(os.path.join(Root_Directory,"Cancer_VL.csv")))
TE_Data = np.array(pd.read_csv(os.path.join(Root_Directory,"Cancer_TE.csv")))

In [4]:
print("Training dataset shape is : ",np.shape(TR_Data))
print("Validation dataset shape is : ",np.shape(VL_Data))
print("Test dataset shape is : ",np.shape(TE_Data))

Training dataset shape is :  (230, 31)
Validation dataset shape is :  (140, 31)
Test dataset shape is :  (71, 31)


### [2] Normalization 해준다

In [5]:
TR_Input = TR_Data[:,1:]
VL_Input = VL_Data[:,1:]
TE_Input = TE_Data[:,1:]
VL_Target = VL_Data[:,:1]
TE_Target = TE_Data[:,:1]

In [6]:
print("TR_Input : ",np.shape(TR_Input))
print("VL_Input : ",np.shape(VL_Input))
print("TE_Input : ",np.shape(TE_Input))
print("VL_Target : ",np.shape(VL_Target))
print("TE_Target : ",np.shape(TE_Target))

TR_Input :  (230, 30)
VL_Input :  (140, 30)
TE_Input :  (71, 30)
VL_Target :  (140, 1)
TE_Target :  (71, 1)


### [3] Training dataset 기준으로 Normalization 해준다, Test시에서는 TR,VL기준으로 Normalization 해준다

In [7]:
TR_Mean = np.mean(TR_Input,0)
TR_Std = np.std(TR_Input,0)

In [8]:
N_TR_Input = (TR_Input-TR_Mean)/TR_Std
N_VL_Input = (VL_Input-TR_Mean)/TR_Std

In [9]:
TR_VL_Mean = np.mean(np.vstack((TR_Input,VL_Input)),0)
TR_VL_Std = np.std(np.vstack((TR_Input,VL_Input)),0)

In [10]:
N2_TR_Input = (np.vstack((TR_Input,VL_Input))-TR_VL_Mean)/TR_VL_Std
N2_TE_Input = (TE_Input-TR_VL_Mean)/TR_VL_Std

### [4] 각 모델들을 학습 시켜서 Validation dataset에서 최적의 조건을 찾는다

### [4-1] Single Gauss

In [11]:
# 가우시안 분포 추정 함수
def estimateGaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.cov(dataset.T)
    return mu, sigma
    
# 다변량 가우시안에서 pdf를 리턴하는 함수
def multivariateGaussian(dataset,mu,sigma):
    p = multivariate_normal(mean=mu, cov=sigma)
    return p.pdf(dataset)

# Validiation dataset을 통해 가장 좋은 epsilon(임계치) 탐색
def selectThresholdByCV(probs,gt):  
    
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(probs) - min(probs)) / 1000;
    epsilons = np.arange(min(probs),max(probs),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (probs < epsilon) 
        f = f1_score(gt, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon
    
    return best_f1, best_epsilon

In [12]:
mu, sigma = estimateGaussian(N_TR_Input)

In [13]:
np.shape(mu)

(30,)

In [14]:
np.shape(sigma)

(30, 30)

In [15]:
p = multivariateGaussian(N_TR_Input,mu,sigma)

In [16]:
#selecting optimal value of epsilon using validation dataset
p_cv = multivariateGaussian(N_VL_Input,mu,sigma)
fscore, ep = selectThresholdByCV(p_cv,VL_Target[:,0])
print(fscore, ep)

0.8944723618090452 0.38963552265976753


#### Test 데이터에 적용한다

In [17]:
mu, sigma = estimateGaussian(N2_TR_Input)

In [18]:
p_Guass_TE = multivariateGaussian(N2_TE_Input,mu,sigma)

In [19]:
#selecting outlier datapoints 
outliers_Gauss = (p_Guass_TE < ep)+0

In [20]:
outliers_Gauss

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1])

In [21]:
f1_score(TE_Target, outliers_Gauss,average='binary')

0.7826086956521738

### [4-2] Mixture of Gaussian

In [22]:
from sklearn import mixture
GMM5 = mixture.GaussianMixture(n_components=5,covariance_type='diag',random_state=15,max_iter=500)
GMM10 = mixture.GaussianMixture(n_components=10,covariance_type='diag',random_state=15,max_iter=500)
GMM15 = mixture.GaussianMixture(n_components=15,covariance_type='diag',random_state=15,max_iter=500)
GMM5.fit(N_TR_Input)
GMM10.fit(N_TR_Input)
GMM15.fit(N_TR_Input)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=500,
        means_init=None, n_components=15, n_init=1, precisions_init=None,
        random_state=15, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [23]:
# Validation을 통해 가장 좋은 epsilon(임계치) 탐색
def selectThresholdByGMM(NL,gt):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(NL) - min(NL)) / 1000;
    epsilons = np.arange(min(NL)+0.001,max(NL),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (NL > epsilon) 
        f = f1_score(gt, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon    
    return best_f1, best_epsilon

In [24]:
GMM5_Score = -GMM5.score_samples(N_VL_Input) 
GMM10_Score = -GMM10.score_samples(N_VL_Input) 
GMM15_Score = -GMM15.score_samples(N_VL_Input) 

In [25]:
fscore_GMM5, ep_GMM5 = selectThresholdByGMM(GMM5_Score,VL_Target[:,0])
fscore_GMM10, ep_GMM10 = selectThresholdByGMM(GMM10_Score,VL_Target[:,0])
fscore_GMM15, ep_GMM15 = selectThresholdByGMM(GMM15_Score,VL_Target[:,0])

In [26]:
print("GMM 5 F-score : ",fscore_GMM5)
print("GMM 10 F-score : ",fscore_GMM10)
print("GMM 15 F-score : ",fscore_GMM15)

GMM 5 F-score :  0.9263157894736842
GMM 10 F-score :  0.9263157894736842
GMM 15 F-score :  0.9101123595505618


#### Test 데이터에 적용한다

In [27]:
GMM_TE = mixture.GaussianMixture(n_components=5,covariance_type='diag',random_state=15,max_iter=500)
GMM_TE.fit(N2_TR_Input)
GMM_TE_Score = -GMM_TE.score_samples(N2_TE_Input) 
outliers_GMM = (GMM_TE_Score > ep_GMM5)+0

In [28]:
f1_score(TE_Target, outliers_GMM,average='binary')

0.7804878048780488

### [4-3] Isolation Forest

In [29]:
iforest_50 = IsolationForest(n_estimators=50,random_state=15)
iforest_100 = IsolationForest(n_estimators=100,random_state=15)
iforest_150 = IsolationForest(n_estimators=150,random_state=15)
iforest_200 = IsolationForest(n_estimators=200,random_state=15)
iforest_50.fit(N_TR_Input)
iforest_100.fit(N_TR_Input)
iforest_150.fit(N_TR_Input)
iforest_200.fit(N_TR_Input)

IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
        max_features=1.0, max_samples='auto', n_estimators=200,
        n_jobs=None, random_state=15, verbose=0)

In [30]:
anomaly_score_Valid_50 = iforest_50.decision_function(N_VL_Input)
anomaly_score_Valid_100 = iforest_100.decision_function(N_VL_Input)
anomaly_score_Valid_150 = iforest_150.decision_function(N_VL_Input)
anomaly_score_Valid_200 = iforest_200.decision_function(N_VL_Input)

In [31]:
# Validation set을 통해 가장 좋은 epsilon(임계치) 탐색
def selectThresholdByIF(NL,gt):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(NL) - min(NL)) / 1000;
    epsilons = np.arange(min(NL)+0.001,max(NL),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (NL < epsilon) 
        f = f1_score(gt, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon    
    return best_f1, best_epsilon

In [32]:
fscore_50, threshold_50 = selectThresholdByIF(anomaly_score_Valid_50,VL_Target[:,0])
fscore_100, threshold_100 = selectThresholdByIF(anomaly_score_Valid_100,VL_Target[:,0])
fscore_150, threshold_150 = selectThresholdByIF(anomaly_score_Valid_150,VL_Target[:,0])
fscore_200, threshold_200 = selectThresholdByIF(anomaly_score_Valid_200,VL_Target[:,0])

In [33]:
print("IF 50 F-score : ",fscore_50)
print("IF 100 F-score : ",fscore_100)
print("IF 150 F-score : ",fscore_150)
print("IF 200 F-score : ",fscore_200)

IF 50 F-score :  0.8972972972972973
IF 100 F-score :  0.9090909090909091
IF 150 F-score :  0.9081081081081082
IF 200 F-score :  0.9071038251366121


#### Test 데이터에 적용한다

In [34]:
iforest_100.fit(N2_TR_Input)
anomaly_score_Valid_100 = iforest_100.decision_function(N2_TE_Input)

In [35]:
#selecting outlier datapoints 
outliers_IsolationForest = (anomaly_score_Valid_100 < threshold_100)+0

In [36]:
f1_score(TE_Target, outliers_IsolationForest,average='binary')

0.8181818181818182

### [4-4] Local outlier factor

In [37]:
LoF_10 = LocalOutlierFactor(n_neighbors=10,novelty=True)
LoF_30 = LocalOutlierFactor(n_neighbors=30,novelty=True)
LoF_40 = LocalOutlierFactor(n_neighbors=40,novelty=True)
LoF_50 = LocalOutlierFactor(n_neighbors=50,novelty=True)
LoF_10.fit(N_TR_Input)
LoF_30.fit(N_TR_Input)
LoF_40.fit(N_TR_Input)
LoF_50.fit(N_TR_Input)

LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
          metric='minkowski', metric_params=None, n_jobs=None,
          n_neighbors=50, novelty=True, p=2)

In [38]:
LoF_10_Score = LoF_10.decision_function(N_VL_Input) 
LoF_30_Score = LoF_30.decision_function(N_VL_Input) 
LoF_40_Score = LoF_40.decision_function(N_VL_Input) 
LoF_50_Score = LoF_50.decision_function(N_VL_Input) 

In [39]:
# Validation set을 통해 가장 좋은 epsilon(임계치) 탐색
def selectThresholdByLoF(NL,gt):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    stepsize = (max(NL) - min(NL)) / 1000;
    epsilons = np.arange(min(NL)+0.001,max(NL),stepsize)
    for epsilon in np.nditer(epsilons):
        predictions = (NL < epsilon) 
        f = f1_score(gt, predictions,average='binary')
        if f > best_f1:
            best_f1 = f
            best_epsilon = epsilon    
    return best_f1, best_epsilon

In [40]:
fscore_LoF_10, threshold_LoF_10 = selectThresholdByLoF(LoF_10_Score,VL_Target[:,0])
fscore_LoF_30, threshold_LoF_30 = selectThresholdByLoF(LoF_30_Score,VL_Target[:,0])
fscore_LoF_40, threshold_LoF_40 = selectThresholdByLoF(LoF_40_Score,VL_Target[:,0])
fscore_LoF_50, threshold_LoF_50 = selectThresholdByLoF(LoF_50_Score,VL_Target[:,0])

In [41]:
print("LoF 10 F-score : ",fscore_LoF_10)
print("LoF 30 F-score : ",fscore_LoF_30)
print("LoF 40 F-score : ",fscore_LoF_40)
print("LoF 50 F-score : ",fscore_LoF_50)

LoF 10 F-score :  0.8021978021978022
LoF 30 F-score :  0.9297297297297298
LoF 40 F-score :  0.93048128342246
LoF 50 F-score :  0.93048128342246


#### Test 데이터에 적용한다

In [42]:
LoF_Test = LocalOutlierFactor(n_neighbors=40,novelty=True)
LoF_Test.fit(N2_TR_Input)
LoF_Test_Score = LoF_40.decision_function(N2_TE_Input) 

In [43]:
#selecting outlier datapoints 
outliers_LoF = (LoF_Test_Score < threshold_LoF_40)+0

In [44]:
f1_score(TE_Target, outliers_LoF,average='binary')

0.75

## [5] 모델 별 평가

In [45]:
## Real Value ##
# TE_Target

## Single Gaussian ##
# p_Guass_TE
# outliers_Gauss

## Gaussian mixture ##
# GMM_TE_Score
# outliers_GMM

## Isolation Forest ##
# anomaly_score_Valid_100
# outliers_IsolationForest

## Local outlier factor ##
# LoF_Test_Score
# outliers_LoF

In [46]:
# 평가지표를 위한 함수를 생성한다.
def Valid_Index(Data,NAME):
    Accuracy = (Data[0,0]+Data[1,1])/np.sum(Data)
    TPR = Data[1,1]/np.sum(Data[1,:])
    TNR = Data[0,0]/np.sum(Data[0,:])
    Precision = Data[1,1]/np.sum(Data[:,1])
    BCR = np.sqrt(TPR*TNR)
    F1 = (2*TPR*Precision)/(TPR+Precision)
    TMP=pd.DataFrame({'Model' : NAME,
                  'Accuracy' : [Accuracy],
                  'TPR': [TPR],
                  'TNR': [TNR],
                  'Precision': [Precision],
                  'BCR': [BCR],
                  'F1': [F1]})
    Results=TMP[['Model','Accuracy','F1','BCR','Precision','TPR','TNR']]
    return(Results)

In [47]:
Total_Results = Valid_Index(confusion_matrix(TE_Target,outliers_Gauss),"Single_Gauss").append([
Valid_Index(confusion_matrix(TE_Target,outliers_GMM),"GaussianMixture"),
Valid_Index(confusion_matrix(TE_Target,outliers_IsolationForest),"IsolationForest"),
Valid_Index(confusion_matrix(TE_Target,outliers_LoF),"LocalOutlierFactor")])

Total_Results = Total_Results.sort_values(by=['F1'],ascending=False)
pd.DataFrame(Total_Results)

,Model,Accuracy,F1,BCR,Precision,TPR,TNR
0,IsolationForest,0.887324,0.818182,0.891133,0.750000,0.90,0.882353
0,Single_Gauss,0.859155,0.782609,0.871105,0.692308,0.90,0.843137
0,GaussianMixture,0.873239,0.780488,0.849452,0.761905,0.80,0.901961
0,LocalOutlierFactor,0.859155,0.750000,0.822478,0.750000,0.75,0.901961


In [48]:
from sklearn.metrics import roc_auc_score
print( "Gauss AUROC : ",np.round(roc_auc_score(TE_Target, -p_Guass_TE),3))
print( "GMM AUROC : ",np.round(roc_auc_score(TE_Target, GMM_TE_Score),3))
print( "IF AUROC : ",np.round(roc_auc_score(TE_Target, -anomaly_score_Valid_100),3))
print( "LoF AUROC : ",np.round(roc_auc_score(TE_Target, -LoF_Test_Score),3))

Gauss AUROC :  0.931
GMM AUROC :  0.945
IF AUROC :  0.953
LoF AUROC :  0.937
